## Deliverable 2. Create a Customer Travel Destinations Map.

In [45]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [46]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
0,0,Vardo,70.3705,31.1107,53.80,96,100,8.70,NO,light rain,2021-08-01 21:56:48
1,1,Lavrentiya,65.5833,-171.0000,41.14,84,100,12.68,RU,light rain,2021-08-01 22:00:17
2,2,Rikitea,-23.1203,-134.9692,69.31,58,53,22.48,PF,broken clouds,2021-08-01 21:58:30
3,3,Cabo San Lucas,22.8909,-109.9124,93.61,70,66,1.99,MX,broken clouds,2021-08-01 22:00:17
4,4,Mataura,-46.1927,168.8643,42.48,95,80,2.82,NZ,broken clouds,2021-08-01 21:58:34


In [47]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [48]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description,Date
3,3,Cabo San Lucas,22.8909,-109.9124,93.61,70,66,1.99,MX,broken clouds,2021-08-01 22:00:17
11,11,Salalah,17.0151,54.0924,75.38,92,100,5.91,OM,overcast clouds,2021-08-01 21:59:06
15,15,Atuona,-9.8000,-139.0333,77.11,69,31,11.18,PF,scattered clouds,2021-08-01 21:58:33
19,19,Kropotkin,45.4375,40.5756,78.21,39,61,4.85,RU,broken clouds,2021-08-01 22:00:23
20,20,Yellowknife,62.4560,-114.3525,87.33,28,1,1.99,CA,clear sky,2021-08-01 22:00:24
21,21,Lafayette,30.2335,-92.0668,92.91,71,80,4.00,US,broken clouds,2021-08-01 22:00:25
23,23,Rudbar,30.1500,62.6000,85.64,15,0,19.22,AF,clear sky,2021-08-01 21:59:05
25,25,Dodge City,37.7528,-100.0171,85.24,41,76,11.99,US,broken clouds,2021-08-01 22:00:27
26,26,Lompoc,34.6391,-120.4579,90.12,64,20,8.99,US,few clouds,2021-08-01 21:58:00
30,30,Avarua,-21.2078,-159.7750,75.25,94,75,12.66,CK,light rain,2021-08-01 21:57:21


In [49]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        212
City           212
Lat            212
Lng            212
Max Temp       212
Humidity       212
Cloudiness     212
Wind Speed     212
Country        212
Description    212
Date           212
dtype: int64

In [50]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [51]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Cabo San Lucas,MX,93.61,broken clouds,22.8909,-109.9124,
11,Salalah,OM,75.38,overcast clouds,17.0151,54.0924,
15,Atuona,PF,77.11,scattered clouds,-9.8000,-139.0333,
19,Kropotkin,RU,78.21,broken clouds,45.4375,40.5756,
20,Yellowknife,CA,87.33,clear sky,62.4560,-114.3525,
21,Lafayette,US,92.91,broken clouds,30.2335,-92.0668,
23,Rudbar,AF,85.64,clear sky,30.1500,62.6000,
25,Dodge City,US,85.24,broken clouds,37.7528,-100.0171,
26,Lompoc,US,90.12,few clouds,34.6391,-120.4579,
30,Avarua,CK,75.25,light rain,-21.2078,-159.7750,


In [52]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [53]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Cabo San Lucas,MX,93.61,broken clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
11,Salalah,OM,75.38,overcast clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
15,Atuona,PF,77.11,scattered clouds,-9.8000,-139.0333,Villa Enata
19,Kropotkin,RU,78.21,broken clouds,45.4375,40.5756,Hotel Europe Kropotkin
20,Yellowknife,CA,87.33,clear sky,62.4560,-114.3525,The Explorer Hotel
...,...,...,...,...,...,...,...
567,Pataskala,US,78.62,few clouds,39.9956,-82.6743,Rona Homes
573,Umm Lajj,SA,87.58,clear sky,25.0213,37.2685,Moon light Furnished Units
574,Bairiki,KI,82.33,broken clouds,1.3292,172.9752,Tarawa Boutique Hotel
576,Lagunillas,VE,88.65,few clouds,10.1301,-71.2595,Casa de Beatriz y Chucho


In [58]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0,how="any")

In [55]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [57]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(0, 0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=100, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))